In [1]:
import io, os, sys, types
import numpy as np

In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import datetime

In [3]:
from implementations import *
from proj1_helpers import *
from cross_validation import *

In [4]:
def standardize(x):
    ''' fill your code in here...
    '''
    centered_data = x - np.mean(x, axis=0)
    std_data = centered_data / np.std(centered_data, axis=0)
    
    return std_data

In [5]:
def subset_test_prediction(x_te, w_star, y_te):
    '''
    Give the accuracy over a test subset data from train
    '''
    #Same method competition (% accuracy)
    #test prediction x_te 
    y_pred = predict_labels(w_star, x_te)
    #compare y_pred,y_test
    s = 0
    for i in range(len(y_te)):
        if y_te[i] == y_pred[i]:
            s+=1
    print("Accuracy:"+str(s/len(y_te))+"%")

In [6]:
def prediction(w,tX,ids_test):
    '''
    Give the .csv file for competition
    '''
    OUTPUT_PATH = 'data/predictions.csv' 
    y_pred = predict_labels(w, tX)
    create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [7]:
def generate_initial_w(shape):
    return np.random.normal(0., 0.1, [shape,])

In [8]:
DATA_TRAIN_PATH = 'data/train.csv' 
DATA_TEST_PATH = 'data/test.csv' 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
y_train, tX_train, ids_train = load_csv_data(DATA_TRAIN_PATH)
tX_train_std = standardize(tX_train)

# Test GD

In [9]:
# Define the parameters of the algorithm.
max_iters = 100
gamma = 0.1
ratio = 0.3

def pred_GD(max_iters,gamma,tX,y,seed,ratio):
    x_tr,x_te,y_tr,y_te = split_data(tX, y, 0.3, seed)

    # Initialization
    w_initial = np.random.rand(30)

    # Start gradient descent.
    w, loss = mean_squared_error_gd(y_tr, x_tr, w_initial, max_iters, gamma)

    if ratio==1:
        #Create the .csv file
        _, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
        prediction(w,tX,ids_test)
    else :
        #Give accuracy on the test subset
        subset_test_prediction(x_te,w,y_te)



In [10]:
pred_GD(max_iters,gamma,tX_train_std,y_train,1,ratio)

Accuracy:0.7119485714285714%


# Cross validation(using Ridge regression), building data

In [11]:
#parameter
degrees = np.arange(2,4)
k_fold = 3
lambdas = np.logspace(-4, 0, 30)
seed = 1

#split data train/test
x_tr,x_te,y_tr,y_te = split_data(tX_train_std, y_train, 0.3, seed)

best_degree, best_lambda, best_rmse = best_degree_selection(x_tr,y_tr,degrees, k_fold, lambdas, seed = 1)
print(best_degree, best_lambda, best_rmse)

3 [0.0001] 0.7975600393919103


In [12]:
#TEST intern substest
poly_tr = build_poly(x_tr, best_degree)
poly_te = build_poly(x_te, best_degree)
w_star, ridge_loss = ridge_regression(y_tr,poly_tr, best_lambda)
subset_test_prediction(poly_te,w_star,y_te)

Accuracy:0.7744742857142857%


In [13]:
poly_te.shape,w_star.shape,y_te.shape,x_te.shape

((175000, 90), (90,), (175000,), (175000, 30))

###  1 )Idea : test GD with ridge regression method(lambda), then Cross valid with GD ridge regression
###  2) test same method with SGD
###  3) test same method with MAE loss instead of MSE

In [14]:
def accuracy(labels_gt, labels_pred):
    """ Computes accuracy.
    
    Args:
        labels_gt (np.array): GT labels of shape (N, ).
        labels_pred (np.array): Predicted labels of shape (N, ).
        
    Returns:
        float: Accuracy, in range [0, 1].
    """
    np.sum(np.abs(labels_gt - labels_pred)==0)
    
    return np.sum(labels_gt == labels_pred) / labels_gt.shape[0]

In [15]:
initial_w = generate_initial_w(tX_train.shape[1])
w_rlr, loss_rlr = reg_logistic_regression(y_train, tX_train_std, 0, initial_w, 1000, 0.001)

In [16]:
predictions = predict_labels(w_rlr, tX_train_std)

In [17]:
accuracy(y_train, predictions)

0.646672

In [18]:
bad_variables = np.where(tX_train == -999)

means_without_outliers = np.zeros(tX_train.shape[1])
counts_without_outliers = np.zeros(tX_train.shape[1])

for entry in tX_train:
    for id_point, point in enumerate(entry):
        if(point != -999):
            means_without_outliers[id_point] += point
            counts_without_outliers[id_point] += 1
            
means_without_outliers /= counts_without_outliers

tX_train_no_outliers = tX_train.copy()
for entry, point in zip(bad_variables[0], bad_variables[1]):
    tX_train_no_outliers[entry, point] = means_without_outliers[point]
    
tX_train_no_outliers_standardized = standardize(tX_train_no_outliers)

In [19]:
initial_w_2 = generate_initial_w(tX_train_no_outliers_standardized.shape[1])
w_rlr_2, loss_rlr_2 = reg_logistic_regression(y_train, tX_train_no_outliers_standardized, best_lambda, initial_w_2, 1000, 0.001)